In [1]:
# Librerias
import os
import re
import time
import glob
import json
import random
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
archivos = ["Consolidado.parquet", "DataMensualContrato.parquet"]

for archivo in archivos:
    if os.path.exists(archivo):
        os.remove(archivo)
        print(f"Archivo {archivo} eliminado correctamente.")
    else:
        print(f"El archivo {archivo} no existe.")

Archivo Consolidado.parquet eliminado correctamente.
Archivo DataMensualContrato.parquet eliminado correctamente.


In [3]:
rutaOece = os.path.join(os.getcwd(), 'RegistroOece')
os.makedirs(rutaOece, exist_ok=True)

patron = re.compile(r'procesado_(\d+)\.html')
RucsProcesados = []

for archivo in os.listdir(rutaOece):
    if archivo.endswith('.html'):
        match = patron.match(archivo)
        if match:
            numero = int(match.group(1))
            RucsProcesados.append(numero)

RucsProcesados = [str(ruc) for ruc in RucsProcesados] 

In [4]:
def SinDuplicados(ruta_archivo):
    if os.path.exists(ruta_archivo):
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            lineas = [line.strip() for line in f if line.strip()]
        
        lineas_unicas = sorted(set(lineas))
        
        with open(ruta_archivo, 'w', encoding='utf-8') as f:
            for linea in lineas_unicas:
                f.write(f"{linea}\n")

SinDuplicados("RucsImposibles.txt")

In [5]:
archivo_rucs = "ListaRucs.xlsx"
archivo_imposibles = 'RucsImposibles.txt'

with open(archivo_imposibles, 'r', encoding='utf-8') as f:
    RucsImposibles = [line.strip() for line in f if line.strip()]

DataRucs = pd.read_excel(archivo_rucs)
Rucs = DataRucs['RUC'].astype(str).str.strip()

Rucs = Rucs[~Rucs.isin(RucsImposibles)]
print(f"Faltan procesar {len(Rucs)} RUCs")

Faltan procesar 117 RUCs


In [6]:
if True:
    RucsProcesados = [str(ruc).strip() for ruc in RucsProcesados]
    Rucs = Rucs[~Rucs.isin(RucsProcesados)]
    #Rucs = Rucs.head(1)
    print(f"Faltan procesar {len(Rucs)} RUCs")

Faltan procesar 1 RUCs


In [7]:
def registrar_ruc_imposible(ruc, archivo=archivo_imposibles):
    with open(archivo, 'a', encoding='utf-8') as f:
        f.write(f"{ruc}\n")

if len(Rucs) != 0:

    driver = webdriver.Chrome()

    try:
        for ruc in Rucs:
            ruc = str(ruc).strip()

            URL = f"https://eap.oece.gob.pe/perfilprov-bus/1.0/ficha/{ruc}/contrataciones/exportar"
            try:
                driver.get(URL)
                time.sleep(10) 
                html_completo = driver.page_source

            except WebDriverException as e:
                print(f"Error cargando la página para el RUC {ruc}: {e}")
                registrar_ruc_imposible(ruc)
                continue

            nombre_archivo = f'procesado_{ruc}.html'
            ruta_archivo = os.path.join(rutaOece, nombre_archivo)

            hits_all_past = 0
            if ruc in RucsProcesados and os.path.exists(ruta_archivo):
                try:
                    with open(ruta_archivo, "r", encoding="utf-8") as f:
                        html_pasado = BeautifulSoup(f.read(), "html.parser")
                    json_text_past = html_pasado.find("pre").text
                    datos_pasados = json.loads(json_text_past)
                    if datos_pasados.get("searchInfo") and datos_pasados["searchInfo"] is not None:
                        hits_all_past = datos_pasados["searchInfo"].get("hitsAll", 0)
                except Exception as e:
                    print(f"Error leyendo archivo pasado para {ruc}: {e}")

            html_presente = BeautifulSoup(html_completo, "html.parser")
            pre_tag = html_presente.find("pre")

            if not pre_tag:
                print(f"No se encontró el tag <pre> en el HTML para el RUC: {ruc}, no se procesa.")
                registrar_ruc_imposible(ruc)
                continue

            try:
                json_text_now = pre_tag.text
                datos = json.loads(json_text_now)
            except Exception as e:
                print(f"Error parseando JSON para el RUC {ruc}: {e}")
                registrar_ruc_imposible(ruc)
                continue

            search_info = datos.get("searchInfo")
            if search_info is None:
                print(f"No hay datos útiles para el RUC: {ruc}, no se guarda el HTML.")
                registrar_ruc_imposible(ruc)
                continue

            hits_all_now = search_info.get("hitsAll", 0)

            if hits_all_now == 0:
                print(f"No hay resultados (hitsAll == 0) para el RUC: {ruc}")
                registrar_ruc_imposible(ruc)
                continue

            if hits_all_past != hits_all_now and hits_all_past == 0 and hits_all_now != 0:
                try:
                    with open(ruta_archivo, 'w', encoding='utf-8') as f:
                        f.write(html_completo)
                    print(f"Guardado HTML para el RUC: {ruc}")
                except Exception as e:
                    print(f"Error guardando archivo para el RUC {ruc}: {e}")
                    registrar_ruc_imposible(ruc)

    finally:
        try:
            driver.quit()
        except Exception as e:
            print(f"Error cerrando el navegador: {e}")

Guardado HTML para el RUC: 20527026238


In [8]:
lista_dfs = []

for archivo in os.listdir(rutaOece):
    if archivo.endswith('.html'):
        ruta_archivo = os.path.join(rutaOece, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            data_json = json.loads(soup.find('pre').text)
            if 'contratosE01' in data_json:
                df = pd.DataFrame(data_json['contratosE01'])
                ruc = archivo.replace('procesado_', '').replace('.html', '')
                df['RUC Origen'] = ruc
                lista_dfs.append(df)

df_consolidado = pd.concat(lista_dfs, ignore_index=True)
df_consolidado = df_consolidado.drop_duplicates()

In [9]:
df_consolidado['miembros_consorcio'] = df_consolidado['miembros_consorcio'].fillna('')  # reemplaza NaN por ''
mask_vacio = df_consolidado['miembros_consorcio'].str.strip() == ''  # filas con cadena vacía o espacios
df_consolidado.loc[mask_vacio, 'miembros_consorcio'] = df_consolidado.loc[mask_vacio, 'RUC Origen'].astype(str) + "|100"

In [10]:
def obtener_porcentaje(ruc_objetivo, miembros):
    if pd.isna(miembros):
        return None
    pares = miembros.split('||')
    for par in pares:
        partes = par.split('|')
        if len(partes) >= 2 and partes[0] == ruc_objetivo:
            try:
                return float(partes[1])
            except ValueError:
                return None
    return None

df_consolidado['monto_del_contrato_original'] = pd.to_numeric(
    df_consolidado['monto_del_contrato_original'], errors='coerce')

df_consolidado['participacion_ruc_origen'] = df_consolidado.apply(
    lambda row: obtener_porcentaje(row['RUC Origen'], row.get('miembros_consorcio')),
    axis=1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['participacion_ruc_origen'] * df_consolidado['monto_del_contrato_original'] /100

df_consolidado['fecha_de_firma_de_contrato'] = pd.to_datetime(df_consolidado['fecha_de_firma_de_contrato'], dayfirst=True, errors='coerce')
df_consolidado['fecha_prevista_de_fin_de_contrato'] = pd.to_datetime(df_consolidado['fecha_prevista_de_fin_de_contrato'], dayfirst=True, errors='coerce')

df_consolidado['Nro de dias'] = (
    df_consolidado['fecha_prevista_de_fin_de_contrato'] - df_consolidado['fecha_de_firma_de_contrato']).dt.days

df_consolidado['Valor Mensual proporcional'] = (df_consolidado['Valor Proporcional GE']/df_consolidado['Nro de dias'])*30

df_consolidado['plazo en Meses'] = round(df_consolidado['Nro de dias'] /30,1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['Valor Proporcional GE'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

df_consolidado['Valor Mensual proporcional'] = df_consolidado['Valor Mensual proporcional'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

In [11]:
df_consolidado.rename(columns={
    'objeto': 'Objeto',
    'descripcion': 'Descripción',
    'entidad': 'Entidad',
    'moneda_del_monto_del_contrato_original': 'Moneda del Contrato Original',
    'monto_del_contrato_original': 'Monto del Contrato Original',
    'fecha_de_firma_de_contrato': 'Fecha de Firma de Contrato',
    'fecha_prevista_de_fin_de_contrato': 'Fecha Prevista de FIn de Contrato',
    'miembros_consorcio': 'Miembros Consorcio',
    'estado': 'Estado',
    'RUC Origen': 'RUC',
    'participacion_ruc_origen': '% Participación RUC',
    'Valor Proporcional GE': 'Valor Proporcional GE',
    'Nro de dias': 'N° Días',
    'Valor Mensual proporcional': 'Valor Mensual proporcional',
    'plazo en Meses': 'Plazo en Meses'
}, inplace=True)


In [12]:
patron = re.compile(r'procesado_(\d+)\.html')
RucsProcesados = []

for archivo in os.listdir(rutaOece):
    if archivo.endswith('.html'):
        match = patron.match(archivo)
        if match:
            numero = int(match.group(1))
            RucsProcesados.append(numero)

RucsProcesados = [str(ruc) for ruc in RucsProcesados] 

In [13]:
Ruclist = []
RazonSocialist = []

for ruc in RucsProcesados:
    ruc = str(ruc).strip()
    
    nombre_archivo = f'procesado_{ruc}.html'
    ruta_archivo = os.path.join(rutaOece, nombre_archivo)
    
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        html_pasado = BeautifulSoup(f.read(), "html.parser")

    pre_tag = html_pasado.find("pre")
    if not pre_tag:
        print(f"No se encontró <pre> para el RUC: {ruc}")
        continue

    try:
        json_text_past = pre_tag.text
        datos_pasados = json.loads(json_text_past)

        razon_social = datos_pasados.get("proveedorE02", {}).get("razon_social_proveedor", "")

        Ruclist.append(ruc)
        RazonSocialist.append(razon_social)

    except json.JSONDecodeError as e:
        print(f"Error parseando JSON para {ruc}: {e}")

df_final = {
    'RUC': Ruclist,
    'Razón Social': RazonSocialist}

df_final = pd.DataFrame(df_final)

In [14]:
df_consolidado['RUC'] = df_consolidado['RUC'].astype(str).str.strip()
df_final['RUC'] = df_final['RUC'].astype(str).str.strip()

df_consolidado = pd.merge(
    df_consolidado,
    df_final[['RUC', 'Razón Social']],
    on='RUC',
    how='left')

In [15]:
df_consolidado['RUC'] = df_consolidado['RUC'].astype(str).str.strip()

df_consolidado['Fecha de Firma de Contrato'] = pd.to_datetime(df_consolidado['Fecha de Firma de Contrato'], errors='coerce')
df_consolidado['Fecha Prevista de FIn de Contrato'] = pd.to_datetime(df_consolidado['Fecha Prevista de FIn de Contrato'], errors='coerce')

df_consolidado['% Participación RUC'] = pd.to_numeric(df_consolidado['% Participación RUC'], errors='coerce')

df_consolidado['Valor Proporcional GE'] = df_consolidado['Valor Proporcional GE'].astype(str).str.replace(',', '')
df_consolidado['Valor Proporcional GE'] = pd.to_numeric(df_consolidado['Valor Proporcional GE'], errors='coerce')

df_consolidado['Monto del Contrato Original'] = df_consolidado['Monto del Contrato Original'].astype(str).str.replace(',', '')
df_consolidado['Monto del Contrato Original'] = pd.to_numeric(df_consolidado['Monto del Contrato Original'], errors='coerce')

df_consolidado['N° Días'] = round(df_consolidado['N° Días'],2)

In [16]:
cols = ['RUC', 'Razón Social'] + [col for col in df_consolidado.columns if col not in ['RUC', 'Razón Social']]
df_consolidado = df_consolidado[cols]

In [17]:
df_consolidado.to_parquet("Consolidado.parquet", index=False)

In [18]:
df_consolidado['Valor Mensual proporcional'] = pd.to_numeric(
    df_consolidado['Valor Mensual proporcional'].astype(str).str.replace(',', ''),
    errors='coerce'
).fillna(0)

fecha_inicio = df_consolidado['Fecha de Firma de Contrato'].min()
fecha_fin = df_consolidado['Fecha Prevista de FIn de Contrato'].max()
fechas_periodos = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')

registro = []

for periodo in fechas_periodos:
    contratos_activos = df_consolidado[
        (df_consolidado['Fecha de Firma de Contrato'] <= periodo) &
        (df_consolidado['Fecha Prevista de FIn de Contrato'] >= periodo)
    ].copy()
    
    contratos_activos['Fecha Periodo'] = periodo
    contratos_activos = contratos_activos[['RUC', 'Razón Social', 'Fecha Periodo', 'Valor Mensual proporcional', 'Descripción']]
    
    registro.append(contratos_activos)

df_mensual = pd.concat(registro, ignore_index=True)

df_resumen = df_mensual.groupby(
    ['RUC', 'Razón Social', 'Fecha Periodo'],
    as_index=False
)['Valor Mensual proporcional'].sum()

df_conteo = df_mensual.groupby(['RUC', 'Fecha Periodo'])['Descripción'].nunique().reset_index()
df_conteo = df_conteo.rename(columns={'Descripción': 'Contratos'})

df_resumen = pd.merge(df_resumen, df_conteo, on=['RUC', 'Fecha Periodo'], how='left')
df_resumen['Contratos'] = df_resumen['Contratos'].fillna(0).astype(int)

df_resumen['Valor Mensual proporcional'] = df_resumen['Valor Mensual proporcional'].fillna(0)
df_resumen.to_parquet("DataMensualContrato.parquet", index=False)

C:\Users\eholgado\AppData\Local\Temp\ipykernel_23840\2987101929.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_periodos = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')
